In [1]:
## SETUP:
from trans_tools import *
from IrisBackendv3.codec.payload import CommandPayload
from IrisBackendv3.codec.packet import IrisCommonPacket
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic
from IrisBackendv3.utils.basic import bytearray_to_spaced_hex as hexstr

wired_seq_num = 0x01
wireless_seq_num = wired_seq_num
# steps = ['heater-test','setup', 'power-on', 'wifi-mode', 'deploy', 'misc-test']
itr = 2

send_data_packet_to_wd_before_sniffing = False
specific_cmd_name_override = None


# specific_cmd_name_override = 'power-on-radio'
# specific_cmd_name_override = 'power-on-herc'
# specific_cmd_name_override = 'reset-radio'
# specific_cmd_name_override = 'reset-herc'
# specific_cmd_name_override = 'power-on-motors'

# specific_cmd_name_override = 'setup'
# specific_cmd_name_override = 'power-on'
# specific_cmd_name_override = 'power-off-herc'
# specific_cmd_name_override = 'power-off-radio'

# specific_cmd_name_override = '3v3-on'
# specific_cmd_name_override = '3v3-off'

# specific_cmd_name_override = '24-on'
# specific_cmd_name_override = '24-off'

# specific_cmd_name_override = 'power-off-fpga'
# specific_cmd_name_override = 'power-on-fpga'
# specific_cmd_name_override = 'take-image-0'

# specific_cmd_name_override = 'power-on-motors'
# specific_cmd_name_override = 'reset-motors'
# specific_cmd_name_override = 'drive-fwd-200'
# specific_cmd_name_override = 'motor-control-get-telem'
# specific_cmd_name_override = 'motor-control-spin-one'


In [2]:
## SETTINGS:

serial_device = '/dev/ttyUSB1'
ip="192.168.1.2"
port=8080

In [3]:
settings['SAVE_FILE_PREFIX'] = 'iris__board_3__motor_testing_3-29_2' # this is the prefix on all log files. make it something unique.
load_cache()

In [4]:
## SET TO TRUE TO VIEW A FULL-LIST OF ALL COMMANDS AND TELEMETRY:
module_to_lookup = 'Navigation'

if show_commands := False:
    standards.print_overview()
if show_module_standards := False:
        def module(x): return cprint(f"\n\t{x}", 'magenta', 'on_grey', attrs=['bold'])
        def header(x): return cprint(f"\n\t\t{x}", 'grey', 'on_white')
        def command(i, x): return cprint(f"\n\t\t\t{i}.\t{x}", 'green')
        def telemetry(i, x): return cprint(f"\n\t\t\t{i}.\t{x}", 'red')
        def event(i, x): return cprint(f"\n\t\t\t{i}.\t{x}", 'blue')

        def p_arg(a): return cprint(f"\n\t\t\t\t\tAvailable values for `{a.name}`:", 'cyan')

        def p_enum(x): return cprint(f"\n\t\t\t\t\t\t'{x.name}' or {x.value} or {hex(x.value)}", 'magenta')

        print("Data Standards Overview: [")
        m = standards.modules[module_to_lookup]
        module(m)
        header('Commands:')
        for i, c in enumerate(m.commands.vals):
            command(i, c)
            for arg in c.args:
                if len(arg.enum) > 0:
                    p_arg(arg)
                    for e in arg.enum:
                        p_enum(e)
        header('Telemetry:')
        for i, t in enumerate(m.telemetry.vals):
            telemetry(i, t)
        header('Events:')
        for i, ev in enumerate(m.events.vals):
            event(i, ev)
        print('\n]')

## **Testing Procedure:** picking pre-prepared commands

**Running the Code:**

- On the first run **OR** after resetting / power-cycling the board, **pick a step in the next cell** then press the `red square`, then `green circle`, then `double arrows` to run all the code.
- After changing to the next step (in the next cell), **click here** (yes, this text cell that you're currently reading) and then **press only** the `red square` then the `play button with the down arrow` to send the new command without resetting the sequence numbers.
- You can read wired telemetry streams at the bottom of this notebook and check for the presence of wireless telemetry in wireshark using the filter: `udp && udp.port==8080 && not icmp` . 

**Steps:**

0. Run `setup`, wait for watchdog heartbeats to verify that the mode is now `RS_SERVICE`.
1. Run `power-on`, verify Hercules has been turned on by the presense of `ICP` telemetry packets in the wired output.
2. Run `wifi-mode` to tell Hercules to downlink telemetry wirelessly instead of over the wire. Verify that `ICP` telemetry packets are now showing up in the wireless output. Note: there should no longer be a wired output showing. Also note: wired and wireless commands will still work until the wired connection has been physically disconnected.
3. Run `deploy` to tell Hercules to tell the Watchdog to release its deployment interlock and then have Hercules release its own interlock. Verify that both interlocks are released and that the deployment switch is now closed by removing the deployment power leads from the deployment power supply and verifying that the resistance between them is on the order of single digit Ohms instead of Megaohms.

Once `deploy` has been successfully issued, applying deployment power to the rover will cause its deployment. **Note:** to properly simulate mission conditions, the interlocks should be released and deployment switch closed **before** applying deployment power.

**Troubleshooting:**

**Note:** If you have issues running the `deploy` command, you can instead run the `deploy-wd-only` and the `deploy-herc-only` commands to directly tell the Watchdog and Hercules to release their interlocks separately.

**Note:** If you're having trouble getting wireless comms working, make sure this computer's IP matches the `spacecraft_ip` used by Hercules (usually `192.168.1.120`) and that the IP you're sending to (configured at the top) matches the `rover_ip` used by the Hercules.

In [5]:
## Pick the step you want to run:

steps = ['heater-test', 'setup', 'power-on', 'wifi-mode', 'deploy', 'misc-test']
if specific_cmd_name_override is not None:
    step = specific_cmd_name_override
else:
    step = steps[itr]
step

'power-on'

In [6]:
## Pre-prepared commands list:

'''ACCIDENTAL PASTE'''

source = DataSource.GENERATED

prepared_commands = {
    'setup': ( # Tell the Watchdog to switch into service mode
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_SwitchToServiceMode',
        dict(confirm='CONFIRM_SERVICE'),
        DataPathway.WIRED
    ),
    'power-on': ( # Tell the Watchdog to switch into service mode
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_PrepareForDeployment',
        dict(confirm='CONFIRM_PREP'),
        DataPathway.WIRED
    ),
    'wifi-mode': ( # Turn Everything On
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'GroundInterface_SetPrimaryInterface',
        dict(primary_interface='WF_121'),
        DataPathway.WIRELESS
    ),
    'wired-mode': ( # Turn Everything On
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'GroundInterface_SetPrimaryInterface',
        dict(primary_interface='WATCHDOG'),
        DataPathway.WIRELESS
    ),
    'deploy': (
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'WatchDogInterface_DisengageFromLander',
        dict(confirm='CONFIRM_DEPLOY'),
        DataPathway.WIRED
    ), # Note: there's no way to do just the hercules (you can tell just the hercules but then it'll immediately make watch)
    'deploy-wifi': (
        DataPathway.WIRELESS,
        Magic.COMMAND, # "normal" command is for Hercules
        'WatchDogInterface_DisengageFromLander',
        dict(confirm='CONFIRM_DEPLOY'),
        DataPathway.WIRELESS
    ),
    'deploy-wd-only': ( # special command to tell only WD to release its interlock (in case Herc-WD comms are broken)
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_DisengageFromLander',
        dict(confirm='CONFIRM_DEPLOY'),
        DataPathway.WIRELESS
    ),
    'undeploy': (
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='HDRM_OFF'),
        DataPathway.WIRED
    ), # Note: Deploy2 (Herc deploy pin) does not undeploy, but WD does.
    'undeploy-2': (
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'WatchDogInterface_EngageFromLander',
        dict(),
        DataPathway.WIRED
    ), # Note: Also does not turn off Deploy2
    'heater-test': (
        DataPathway.WIRED,
        Magic.COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='ENABLE_HEATER_CONTROL'),
        DataPathway.WIRELESS
    ),
    'reset-herc': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RESET_HERCULES'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-on-herc': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='HERCULES_POWER_ON'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-off-herc': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='HERCULES_POWER_OFF'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'reset-radio': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RESET_RADIO'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-on-radio': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RADIO_POWER_ON'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-off-radio': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RADIO_POWER_OFF'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'reset-fpga': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RESET_FPGA'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-on-fpga': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='FPGA_POWER_ON'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-off-fpga': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='FPGA_POWER_OFF'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'reset-motors': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='RESET_ALL_MOTORS'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-on-motors': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='ALL_MOTORS_ON'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'power-off-motors': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='ALL_MOTORS_OFF'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    # Navigation_NavDriveForward[distance: uint8, speed: uint8, callback_id: uint16]
    'drive-fwd-200': (
        DataPathway.WIRED,
        Magic.COMMAND,
        'Navigation_NavDriveForward',
        dict(distance=200, speed=100, callback_id=0xBEEF),
        DataPathway.WIRED
    ),
    # Navigation_NavDriveForward[distance: uint8, speed: uint8, callback_id: uint16]
    'motor-control-get-telem': (
        DataPathway.WIRED,
        Magic.COMMAND,
        'MotorControl_McUpdateTelemetry',
        dict(),
        DataPathway.WIRED
    ),
    # Navigation_NavDriveForward[distance: uint8, speed: uint8, callback_id: uint16]
    'motor-control-spin-one': (
        DataPathway.WIRED,
        Magic.COMMAND,
        'MotorControl_McSpin',
        dict(motor_id=0x00, raw_ticks=20000), # Change this to whatever motor you want to control
        DataPathway.WIRED
    ),
    'take-image-0': (
        DataPathway.WIRED,
        Magic.COMMAND,
        'Camera_TakeImage',
        dict(camera_num=0, callback_id=0), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'take-image-1': (
        DataPathway.WIRED,
        Magic.COMMAND,
        'Camera_TakeImage',
        dict(camera_num=1, callback_id=0), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    '3v3-on': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='EN_3_3_POWER_ON'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    '3v3-off': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='EN_3_3_POWER_OFF'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    '24-on': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='EN_24_ON'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    '24-off': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='EN_24_OFF'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    ),
    'misc-test': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        # dict(reset_value='EN_3_3_POWER_ON'), # Change this to whatever you want to reset.
        # dict(reset_value='HDRM_OFF'), # Change this to whatever you want to reset.
        DataPathway.WIRED
    )
}
pathway, magic, command_name, kwargs, telem_pathway = prepared_commands[step]

In [7]:
## Build Command:

'''ACCIDENTAL PASTE'''

if pathway == DataPathway.WIRED:
    seq_num = wired_seq_num
elif pathway == DataPathway.WIRELESS:
    seq_num = wireless_seq_num

command_payload_type = {
    Magic.WATCHDOG_COMMAND: WatchdogCommandPayload,
    Magic.COMMAND: CommandPayload
}[magic]

module, command = standards.global_command_lookup(command_name)
payloads = PayloadCollection(
    CommandPayload=[
        command_payload_type(
            pathway=pathway,
            source=source,
            magic=magic,
            module_id=module.ID,
            command_id=command.ID,
            args=kwargs
        )
    ],
    TelemetryPayload=[],
    EventPayload=[],
    FileBlockPayload=[]
)
packet = IrisCommonPacket(
    seq_num = seq_num, 
    payloads = payloads
).encode()
print(hexstr(packet)) # 02 07 00 d6 ee ff 00 c0 ec 10 77

01 07 00 d9 ee ff 00 c0 01 10 60


In [8]:
## Send Command:
connect_serial(device = serial_device)
from IrisBackendv3.data_standards import DataStandards
if send_data_packet_to_wd_before_sniffing:
    if pathway == DataPathway.WIRED:
        send_data_wd_serial(packet)
    elif pathway == DataPathway.WIRELESS:
        send_wifi(packet, ip=ip, port=port)

Connection Success!


In [9]:
## Stream Telemetry:
if True or telem_pathway == DataPathway.WIRED:
    stream_data_ip_udp_serial()
elif telem_pathway == DataPathway.WIRELESS:
    pass # Check for wireless telemetry in Wireshark. For parsing help, run: `pyenv exec python parse_pcap.py --help`

RS_MISSION:	[Heat: ON, Ctrl: ON] 	329.2°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
RS_MISSION:	[Heat: ON, Ctrl: ON] 	328.6°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535


2022-03-29 22:51:59 Houston IrisBackendv3.codec.logging[27320] WARNING Had to abort packet parsing due to the following exception: ValueError: 285147136 is not a valid Magic

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/iris/Desktop/Stable_Testing_Repos/Ground Backend/Apps/GroundSoftware/IrisBackendv3/codec/magic.py", line 96, in decode
    magic = cls(contents[0])  # type: ignore
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 309, in __call__
    return cls.__new__(cls, value)
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 600, in __new__
    raise exc
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 584, in __new__
    result = cls._missing_(value)
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 613, in _missing_
    raise ValueError("%r is not a valid %s" % (value, cls.__name__))
ValueError: 285147136 is not a valid Magic



ICP[#10::640]: 	0 T	- 0 E	- 0 B	- 0 C
	 Latest Telemetry: 


2022-03-29 22:52:00 Houston IrisBackendv3.codec.logging[27320] WARNING Had to abort packet parsing due to the following exception: ValueError: 285147136 is not a valid Magic

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/iris/Desktop/Stable_Testing_Repos/Ground Backend/Apps/GroundSoftware/IrisBackendv3/codec/magic.py", line 96, in decode
    magic = cls(contents[0])  # type: ignore
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 309, in __call__
    return cls.__new__(cls, value)
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 600, in __new__
    raise exc
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 584, in __new__
    result = cls._missing_(value)
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 613, in _missing_
    raise ValueError("%r is not a valid %s" % (value, cls.__name__))
ValueError: 285147136 is not a valid Magic



ICP[#11::640]: 	0 T	- 0 E	- 0 B	- 0 C
	 Latest Telemetry: 


...

ICP[#35::638]: 	0 T	- 0 E	- 0 B	- 0 C
	 Latest Telemetry: 


2022-03-29 23:02:13 Houston IrisBackendv3.codec.logging[27320] WARNING Had to abort packet parsing due to the following exception: ValueError: 285147136 is not a valid Magic

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/iris/Desktop/Stable_Testing_Repos/Ground Backend/Apps/GroundSoftware/IrisBackendv3/codec/magic.py", line 96, in decode
    magic = cls(contents[0])  # type: ignore
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 309, in __call__
    return cls.__new__(cls, value)
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 600, in __new__
    raise exc
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 584, in __new__
    result = cls._missing_(value)
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 613, in _missing_
    raise ValueError("%r is not a valid %s" % (value, cls.__name__))
ValueError: 285147136 is not a valid Magic



ICP[#36::638]: 	0 T	- 0 E	- 0 B	- 0 C
	 Latest Telemetry: 


2022-03-29 23:02:14 Houston IrisBackendv3.codec.logging[27320] WARNING Had to abort packet parsing due to the following exception: ValueError: 285147136 is not a valid Magic

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/iris/Desktop/Stable_Testing_Repos/Ground Backend/Apps/GroundSoftware/IrisBackendv3/codec/magic.py", line 96, in decode
    magic = cls(contents[0])  # type: ignore
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 309, in __call__
    return cls.__new__(cls, value)
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 600, in __new__
    raise exc
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 584, in __new__
    result = cls._missing_(value)
  File "/home/iris/.pyenv/versions/3.8.5/lib/python3.8/enum.py", line 613, in _missing_
    raise ValueError("%r is not a valid %s" % (value, cls.__name__))
ValueError: 285147136 is not a valid Magic



ICP[#37::638]: 	0 T	- 0 E	- 0 B	- 0 C
	 Latest Telemetry: 
